# Experiment 1

Data: DailyDialog, Emotion-Stimulus, ISEAR

## Import packages

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from transformers import BertTokenizer, BertConfig,AdamW, BertForSequenceClassification,get_linear_schedule_with_warmup
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import accuracy_score,matthews_corrcoef
from tqdm import tqdm, trange,tnrange,tqdm_notebook
import random
import os
import io
from sklearn.preprocessing import LabelEncoder

## Download data

In [ ]:
emotion_list = ['neutral',
                'anger',
                'disgust',
                'fear',
                'joy',
                'sadness',
                'surprise',
                'shame',
                'guilt']

# DailyDialog
daily_dialog = pd.read_csv('../data/raw-data/dailydialog.csv', encoding='utf-8')
daily_dialog['Emotion'].replace({0:'neutral', 1:'anger', 2:'disgust', 3:'fear', 4:'joy', 5:'sadness', 6:'surprise'}, inplace=True)

# Emotion-Stimulus
emotion_stimulus = pd.read_csv('../data/raw-data/emotion-stimulus.csv', encoding='utf-8')
emotion_stimulus['Emotion'].replace({'happy':'joy', 'sad':'sadness'}, inplace=True)

# ISEAR
isear = pd.read_csv('../data/raw-data/isear.csv', encoding='utf-8')

df = daily_dialog.append(emotion_stimulus).append(isear)

print(df.Emotion.value_counts())

neutral     85572
joy         14445
sadness      2792
anger        2576
surprise     2036
fear         1678
disgust      1515
shame        1200
guilt        1052
Name: Emotion, dtype: int64


## Filter emotions

In [ ]:
emotions_to_remove = ['neutral', 'joy', 'shame', 'guilt']
df_filtered = df[~df['Emotion'].isin(emotions_to_remove)]
print(df_filtered.Emotion.value_counts())

NUM_AVG_SAMPLES = round(sum(df_filtered.Emotion.value_counts()) / len(df_filtered.Emotion.value_counts()))

def get_sampled_df(df, emotions_to_sample, num_samples, rand):
    new_df = pd.DataFrame()
    for emotion in emotions_to_sample:
        new_df = new_df.append(df[df['Emotion'] == emotion].sample(n=num_samples, random_state=rand))

    return new_df

emotions_to_sample = ['neutral', 'joy']
df_sampled = get_sampled_df(df, emotions_to_sample, NUM_AVG_SAMPLES, 1)

df_balanced = df_filtered.append(df_sampled)
print(df_balanced.Emotion.value_counts())

sadness     2792
anger       2576
surprise    2036
fear        1678
disgust     1515
Name: Emotion, dtype: int64
sadness     2792
anger       2576
neutral     2119
joy         2119
surprise    2036
fear        1678
disgust     1515
Name: Emotion, dtype: int64


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.2 MB/s 
     |████████████████████████████████| 61 kB 520 kB/s 
     |████████████████████████████████| 895 kB 42.8 MB/s 
     |████████████████████████████████| 596 kB 46.8 MB/s 
     |████████████████████████████████| 3.3 MB 43.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

SEED = 19

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if device == torch.device("cuda"):
    torch.cuda.manual_seed_all(SEED)

In [ ]:
device = torch.device("cuda")

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df_balanced['label_enc'] = labelencoder.fit_transform(df_balanced['Emotion'])

## Model

In [ ]:
sentences = df_balanced.Text.values
print("Distribution of data based on labels: ",df_balanced.label_enc.value_counts())

MAX_LEN = 256
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=True)
input_ids = [tokenizer.encode(sent, add_special_tokens=True, max_length=MAX_LEN, pad_to_max_length=True) for sent in sentences]
labels = df_balanced.label_enc.values

print("Actual sentence before tokenization: ", sentences[2])
print("Encoded Input from dataset: ", input_ids[2])

attention_masks = []
attention_masks = [[float(i>0) for i in seq] for seq in input_ids]
print(attention_masks[2])

Distribution of data based on labels:  5    2792
0    2576
4    2119
3    2119
6    2036
2    1678
1    1515
Name: label_enc, dtype: int64


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Actual sentence before tokenization:   Really ? I think we play you guys next week . Well , I have to go to practice . See you later . 
Encoded Input from dataset:  [101, 1541, 136, 178, 1341, 1195, 1505, 1128, 3713, 1397, 1989, 119, 1218, 117, 178, 1138, 1106, 1301, 1106, 2415, 119, 1267, 1128, 1224, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
train_inputs,validation_inputs,train_labels,validation_labels = train_test_split(input_ids,labels,random_state=41,test_size=0.1)
train_masks,validation_masks,_,_ = train_test_split(attention_masks,input_ids,random_state=41,test_size=0.1)

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

batch_size = 32
train_data = TensorDataset(train_inputs,train_masks,train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data,sampler=train_sampler,batch_size=batch_size)

validation_data = TensorDataset(validation_inputs,validation_masks,validation_labels)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data,sampler=validation_sampler,batch_size=batch_size)

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-cased", num_labels=7).to(device)

lr = 2e-5
adam_epsilon = 1e-8
epochs = 3
num_warmup_steps = 0
num_training_steps = len(train_dataloader)*epochs

optimizer = AdamW(model.parameters(), lr=lr,eps=adam_epsilon,correct_bias=False) 
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

## Training

In [ ]:
train_loss_set = []
learning_rate = []

model.zero_grad()

for _ in tnrange(1,epochs+1,desc='Epoch'):
  print("<" + "="*22 + F" Epoch {_} "+ "="*22 + ">")
  batch_loss = 0

  for step, batch in enumerate(train_dataloader):
    model.train()
    
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    loss = outputs[0]
    
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()

    optimizer.zero_grad()
    
    batch_loss += loss.item()

  avg_train_loss = batch_loss / len(train_dataloader)

  for param_group in optimizer.param_groups:
    print("\n\tCurrent Learning rate: ", param_group['lr'])
    learning_rate.append(param_group['lr'])
    
  train_loss_set.append(avg_train_loss)
  print(F'\n\tAverage Training loss: {avg_train_loss}')
    

  model.eval()

  eval_accuracy,eval_mcc_accuracy,nb_eval_steps = 0, 0, 0

  for batch in validation_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    with torch.no_grad():
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
    logits = logits[0].to('cpu').numpy()
    label_ids = b_labels.to('cpu').numpy()
    pred_flat = np.argmax(logits, axis=1).flatten()
    labels_flat = label_ids.flatten()
    
    df_metrics = pd.DataFrame({'Epoch': epochs,'Actual_class': labels_flat,'Predicted_class': pred_flat})
    
    tmp_eval_accuracy = accuracy_score(labels_flat,pred_flat)
    tmp_eval_mcc_accuracy = matthews_corrcoef(labels_flat, pred_flat)
    eval_accuracy += tmp_eval_accuracy
    eval_mcc_accuracy += tmp_eval_mcc_accuracy
    nb_eval_steps += 1

  print(F'\n\tValidation Accuracy: {eval_accuracy/nb_eval_steps}')
  print(F'\n\tValidation MCC Accuracy: {eval_mcc_accuracy/nb_eval_steps}')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  if __name__ == '__main__':


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

<====================== Epoch 1 ======================>

	Current Learning rate:  1.3333333333333333e-05

	Average Training loss: 0.9342923175062289

	Validation Accuracy: 0.7597517730496455

	Validation MCC Accuracy: 0.7181404217384589
<====================== Epoch 2 ======================>

	Current Learning rate:  6.666666666666667e-06

	Average Training loss: 0.5304762035893481

	Validation Accuracy: 0.7646276595744681

	Validation MCC Accuracy: 0.7251220131808634
<====================== Epoch 3 ======================>

	Current Learning rate:  0.0

	Average Training loss: 0.3514390684414708

	Validation Accuracy: 0.7695035460992907

	Validation MCC Accuracy: 0.7321128156754781


## Done

In [ ]:
model_save_folder = 'model/'
tokenizer_save_folder = 'tokenizer/'

path_model = F'./exp-1/{model_save_folder}'
path_tokenizer = F'./exp-1/{tokenizer_save_folder}'

!mkdir -p {path_model}
!mkdir -p {path_tokenizer}

model.save_pretrained(path_model)
tokenizer.save_pretrained(path_tokenizer)

model_save_name = 'fineTuneModel.pt'
path = path_model = F'./exp-1/{model_save_folder}/{model_save_name}'
torch.save(model.state_dict(),path);

In [ ]:
model_save_folder = 'model/'
model_save_name = 'fineTuneModel.pt'
path = path_model = F'./exp-1/{model_save_folder}/{model_save_name}'
model = BertForSequenceClassification.from_pretrained("bert-base-cased", num_labels=7).to(device)
model.load_state_dict(torch.load(path))
model.eval()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element